# **Wine Quality**
DSCI 100 Winter T2 Group 006-042

## Introduction:

[insert introduction here]

## Preliminary Exploratory Data Analysis:

In [140]:
# loading libraries
library(tidyverse)
library(tidymodels)
library(gridExtra)

In [141]:
# reading data
data <- read_csv("data/WineQT.csv") %>%
        subset(select = -Id)

Parsed with column specification:
cols(
  `fixed acidity` = col_double(),
  `volatile acidity` = col_double(),
  `citric acid` = col_double(),
  `residual sugar` = col_double(),
  chlorides = col_double(),
  `free sulfur dioxide` = col_double(),
  `total sulfur dioxide` = col_double(),
  density = col_double(),
  pH = col_double(),
  sulphates = col_double(),
  alcohol = col_double(),
  quality = col_double(),
  Id = col_double()
)



[insert units here]

In [181]:
# cleaning and wrangling data
clean_data <- data %>%
              mutate(quality = as_factor(quality))
colnames(clean_data) <- gsub(" ", "_", names(clean_data))

In [182]:
# splitting data
clean_data_split <- initial_split(clean_data, prop = 0.5, strata = quality)
train_data <- training(clean_data_split)
test_data <- testing(clean_data_split)

train_data

fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
7.4,0.700,0.00,1.90,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.8,0.760,0.04,2.30,0.092,15,54,0.9970,3.26,0.65,9.8,5
11.2,0.280,0.56,1.90,0.075,17,60,0.9980,3.16,0.58,9.8,6
7.4,0.660,0.00,1.80,0.075,13,40,0.9978,3.51,0.56,9.4,5
7.9,0.600,0.06,1.60,0.069,15,59,0.9964,3.30,0.46,9.4,5
6.7,0.580,0.08,1.80,0.097,15,65,0.9959,3.28,0.54,9.2,5
6.9,0.400,0.14,2.40,0.085,21,40,0.9968,3.43,0.63,9.7,6
6.3,0.390,0.16,1.40,0.080,11,23,0.9955,3.34,0.56,9.3,5
7.6,0.410,0.24,1.80,0.080,4,11,0.9962,3.28,0.59,9.5,5


In [190]:
# summarizing data
train_data_summary <- test_data %>%
                      select(-quality) %>%
                      map_dfr(mean, na.rm = TRUE)
colnames(train_data_summary) <- paste("mean", colnames(train_data_summary), sep = "_")

train_data_summary

mean_fixed_acidity,mean_volatile_acidity,mean_citric_acid,mean_residual_sugar,mean_chlorides,mean_free_sulfur_dioxide,mean_total_sulfur_dioxide,mean_density,mean_pH,mean_sulphates,mean_alcohol
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8.305779,0.5338792,0.2645884,2.490543,0.08786865,15.53765,45.81786,0.9967017,3.314133,0.6592119,10.43158


## Methods:

[insert methods here]

## Expected Outcomes and Significance:

[insert expected outcomes and significance here]